# Utilizando Pyspark para responder questões de negócio - Uber Dataset

Dataset: https://github.com/fivethirtyeight/uber-tlc-foil-response

Esse conjunto de dados contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova York de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

**O objetivo deste notebook será responder as perguntas de negócios abaixo:**

1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

2- Qual o total de veículos que passaram pela base B02617?

3- Qual o total de corridas por base? Apresente de forma decrescente.

## Carregando o Dataset

In [1]:
uberRDD = sc.textFile('data/uber.csv')

In [2]:
uberRDD.take(5)

['dispatching_base_number,date,active_vehicles,trips',
 'B02512,1/1/2015,190,1132',
 'B02765,1/1/2015,225,1765',
 'B02764,1/1/2015,3427,29421',
 'B02682,1/1/2015,945,7679']

In [3]:
uberRDD.count()

355

## 1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?  

In [9]:
basesRDD = uberRDD.map(lambda x: x[0:6])

In [10]:
basesRDD.take(5)

['dispat', 'B02512', 'B02765', 'B02764', 'B02682']

In [11]:
basesRDD.distinct().count() -1

6

In [12]:
basesRDD.distinct().collect()

['dispat', 'B02765', 'B02682', 'B02598', 'B02512', 'B02764', 'B02617']

### Resposta: 

**Há 6 bases Uber: B02765, B02682, B02598, B02512, B02764, B02617**

## 2- Qual o total de veículos que passaram pela base B02617?

In [13]:
basesRDD.distinct().collect()

['dispat', 'B02765', 'B02682', 'B02598', 'B02512', 'B02764', 'B02617']

In [14]:
#Separando o RDD em coluans
uberLinhas = uberRDD.map(lambda line: line.split(","))

In [15]:
uberLinhas.filter(lambda linha: 'B02617' in linha).count()

59

### Resposta:

**59 veículos passaram pela base B02617**

## 3- Qual o total de corridas por base? De forma decrescente

In [16]:
b02617_RDD = uberLinhas.filter(lambda linha: "B02617" in linha)

In [17]:
uberRDD.first()

'dispatching_base_number,date,active_vehicles,trips'

In [18]:
#Criando um novo RDD, porém sem o cabeçalho
uberRDD2 = sc.textFile("data/uber.csv").filter(lambda line: "base" not in line).map(lambda line:line.split(","))

In [19]:
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [20]:
#Organizando em forma decrescente
uberRDD2.map(lambda kp: (kp[0], int(kp[3])) ).reduceByKey(lambda k,v: k + v).takeOrdered(10, key = lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]

### Resposta: 

- B02764: 1914449
- B02617: 725025
- B02682: 662509
- B02598: 540791
- B02765: 193670
- B02512: 93786